In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6) #set default figure size

In [2]:
def Black_Scholes_Call(S, K, r, q, T, sigma):
    """Black-Scholes call option price.
    
    Args:
        S (float): spot price
        K (float): strike price
        r (float): risk-free interest rate
        q (float): dividend yield
        T (float): time to maturity
        sigma (float): volatility

    Returns:
        float: call option price
    """
    if sigma == 0:
        return max(S * np.exp(- q * T) - K * np.exp(- r * T), 0)
    else:
        d1 = (np.log(S/K) + (r - q + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        return S * np.exp(- q * T) * norm.cdf(d1) - K * np.exp(- r * T) * norm.cdf(d2)

In [19]:
def fair_expense_ratio(D, r, sigma, T, tol=1e-6, max_iter=1000000000):
    lower = 0
    upper = 1
    while np.exp(- r * T) * D + np.exp(- upper * T) * D * Black_Scholes_Call(1, np.exp(upper * T), r, 0, sigma, T) - D > 0:
        upper *= 2
    guess = (lower + upper) / 2
    
    while upper - lower > tol and max_iter > 0:
        diff = np.exp(- r * T) * D + np.exp(- guess * T) * D * Black_Scholes_Call(1, np.exp(guess * T), r, 0, sigma, T) - D
        if diff < 0:
            upper = guess
        else:
            lower = guess
        guess = (lower + upper) / 2
        max_iter -= 1
    return guess

In [11]:
# parameters
D = 1
r = 0.01
sigma = 0.2
T = 1

In [20]:
fair_expense_ratio(D, r, sigma, T, tol=1e-6, max_iter=1000)

0.6584839820861816